## Création d'un RAG en Local sur un ensemble de papier de recherche en Reinforcement Deep Learning 

OpenAI liste les principaux papiers du domaine sur cette page : https://spinningup.openai.com/en/latest/spinningup/keypapers.html
On rajoutera aussi le livre de Sutton et al.: http://incompleteideas.net/book/RLbook2020.pdf


## 1: Scrapping des pdf
### Récupération des liens

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# Récupération des liens des pdfs du site
url = "https://spinningup.openai.com/en/latest/spinningup/keypapers.html"
response = requests.get(url)

pdf_links = []

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    links = [a['href'] for a in soup.find_all('a', class_='reference external') if 'href' in a.attrs]
    print("liens trouvés: ", links)
else:
    print("Erreur lors du téléchargement de la page")

# scrapping dans arkiv et openreview
for link in links:

    if link.startswith("https://arxiv.org/") or link.startswith("https://openreview"):
        a_class,pref = ("abs-button download-pdf","https://arxiv.org") if link.startswith("https://arxiv.org/") else ("note_content_pdf","https://openreview.net")
        r = requests.get(link)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text,'html.parser')
            pdf_link = [a['href'] for a in soup.find_all('a', class_=a_class) if 'href' in a.attrs]
            pdf_links.append(pref+pdf_link[0])
        else:
            print("problem")

    elif  link.endswith(".pdf"):
        pdf_links.append(link)

pdf_links.append("http://incompleteideas.net/book/RLbook2020.pdf")

liens trouvés:  ['https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf', 'https://arxiv.org/abs/1507.06527', 'https://arxiv.org/abs/1511.06581', 'https://arxiv.org/abs/1509.06461', 'https://arxiv.org/abs/1511.05952', 'https://arxiv.org/abs/1710.02298', 'https://arxiv.org/abs/1602.01783', 'https://arxiv.org/abs/1502.05477', 'https://arxiv.org/abs/1506.02438', 'https://arxiv.org/abs/1707.06347', 'https://arxiv.org/abs/1707.02286', 'https://arxiv.org/abs/1708.05144', 'https://arxiv.org/abs/1611.01224', 'https://arxiv.org/abs/1801.01290', 'http://proceedings.mlr.press/v32/silver14.pdf', 'https://arxiv.org/abs/1509.02971', 'https://arxiv.org/abs/1802.09477', 'https://arxiv.org/abs/1707.06887', 'https://arxiv.org/abs/1710.10044', 'https://arxiv.org/abs/1806.06923', 'https://openreview.net/forum?id=ByG_3s09KX', 'https://github.com/google/dopamine', 'https://arxiv.org/abs/1611.02247', 'https://arxiv.org/abs/1710.11198', 'https://arxiv.org/abs/1802.10031', 'https://arxiv.org/abs/1702.08892', 'https:/

### Téléchargement des pdfs

In [3]:
import os
from tqdm import tqdm

In [4]:
if not os.path.isdir("data/"):
    os.mkdir("data/")

for (i,pdf_link) in tqdm(enumerate(pdf_links)):
    path = "data/papier_"+str(i)+ ".pdf"
    
    if not os.path.isfile(path):
        pdf_r = requests.get(pdf_link)
        if pdf_r.status_code == 200:
            with open(path,"wb") as f:
                f.write(pdf_r.content)
    else:
        print("probleme de téléchargement")
    

0it [00:00, ?it/s]

probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
p

89it [00:00, 182.73it/s]

probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement


105it [00:05, 19.66it/s]

probleme de téléchargement
probleme de téléchargement
probleme de téléchargement
probleme de téléchargement


## 2: Importation des données

In [5]:
import pandas as pd

In [24]:
from spacy.lang.en import English
def text_to_sentences(text):
    nlp = English()
    nlp.add_pipe("sentencizer")
    doc = nlp(text)

    sentences = [str(s) for s in list(doc.sents)]
    return sentences

def sentence_list_to_chunk(nb_sentence :  int,s_list : list[str]):
    chunks = [" ".join(s_list[i:i+nb_sentence]) for i in range(0,len(s_list),nb_sentence)]
    return chunks

text = "Même si des études récentes montrent qu’il est possible d’utiliser des résolutions plus fines pour des simulations climatiques, la majorité des modèles de climat utilisent une résolution horizontale de l’ordre de 150 km à 50 km. À ces résolutions, seuls les processus de grande échelle sont décrits par les équations primitives discrétisées. Pour représenter les processus de fine échelle, non explicitement résolus par le modèle, comme la convection profonde, la turbulence ou la microphysique, la démarche classique consiste à utiliser des paramétrisations physiques. Les paramétrisations physiques sont des ensembles d’équations, empiriques ou théoriques, développées à partir d’une combinaison de données issues d’observations, de modèles conceptuels, de résultats de modèles à haute résolution, et/ou d’approches théoriques.Une nouvelle méthode permettant de développer de nouvelles paramétrisations consiste à utiliser des techniques de machine learning, en particulier des réseaux de neurones (NN). Les NN pourraient réduire certains biais connus des paramétrisations physiques. Mais la cohérence physique des paramétrisations NN peut être insuffisante. Cela se traduit alors par l’apparition de biais (et/ou d’explosion numérique) dans les simulations climatiques.Il existe une famille de techniques permettant d’implémenter des contraintes physiques aux NN pendant l’apprentissage, appelée « NNs informés par la physique » (ou « physics informed neural networks », PINNs). Dans ce stage, nous proposons d’explorer ces approches dans le cadre du développement d’une paramétrisation NN pour la convection profonde. Pour ce faire, nous utiliserons la paramétrisation de convection profonde de notre modèle comme cadre idéal de travail. Il s’agira de mettre en place et tester la valeur ajoutée de contraintes physiques, à travers l’utilisation d’une des deux méthodes suivantes : (i) via un terme de pénalité ajouté à la loss pendant l’apprentissage ou (ii) en ajoutant une couche spécifique à la fin du NN dont les paramètres sont imposés manuellement."
text = text + text
s_list = text_to_sentences(text)

print(len(s_list))
chunks = sentence_list_to_chunk(10,s_list)
print(len(chunks))


24
3


In [28]:
import fitz
import pandas as pd

data_path = "data/"
pdf_files = ["data/"+f for f in os.listdir(data_path) if f.endswith('.pdf')]
pdf_dic = {}

def text_formatter(text:str) -> str:
    cleaned_text = text.replace("\n"," ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    pages_and_text = []
    doc = fitz.open(pdf_path)

    for (page_number,page) in enumerate(doc):
        text = page.get_text()
        text = text_formatter(text=text)
        sentences = text_to_sentences(text)
        chunks = sentence_list_to_chunk(10,sentences)
        pages_and_text.append({
            "page_number": page_number,
            "text": text,
            "chunks": chunks,
            "page_char_count": len(text),
            "page_words_count": len(text.split(" ")),
            "page_sentences_count": len(text.split(". ")),
            "page_token_count": len(text)/4,
            "page_chunks_count": len(chunks)
        })
    return pages_and_text

def open_pdfs(pdf_path_list : list[str]) -> list[dict]:
    pdf_and_text = []
    pdfs_pages_text_list = []
    for (pdf_number,pdf_path) in enumerate(pdf_path_list):

        pages_and_text = open_and_read_pdf(pdf_path)
        pdfs_pages_text_list.append(pages_and_text)
        df = pd.DataFrame(pages_and_text)
        dict_mean = df.describe().round(2).loc['mean'].to_dict()

        doc = fitz.open(pdf_path)
        texte_complet = "".join([page.get_text("text") for page in doc])
        texte_complet = text_formatter(texte_complet)
        pdf_and_text.append({
            "pdf_number":pdf_number,
            "pdf_page_count":doc.page_count,
            "pdf_char_count":len(texte_complet),
            "pdf_word_count":len(texte_complet.split(" ")),
            "pdf_sentence_count_raw":len(texte_complet.split(". ")),
            "pdf_token_count": len(texte_complet) /4,
            "page_mean_char_count": dict_mean["page_char_count"],
            "page_mean_words_count": dict_mean["page_words_count"],
            "page_mean_sentences_count": dict_mean["page_sentences_count"],
            "page_mean_token_count": dict_mean["page_token_count"]
        })
    return pdf_and_text,pdfs_pages_text_list

pdf_and_text,pdfs_pages_text_list = open_pdfs(pdf_files)


In [31]:
df = pd.DataFrame(pdfs_pages_text_list[0])
df.describe().round(2)

,page_number,page_char_count,page_words_count,page_sentences_count,page_token_count,page_chunks_count
count,9.00,9.00,9.00,9.00,9.00,9.00
mean,4.00,4000.78,647.33,32.89,1000.19,3.67
std,2.74,561.28,120.60,17.09,140.32,1.80
min,0.00,3183.00,459.00,19.00,795.75,2.00
25%,2.00,3665.00,546.00,22.00,916.25,3.00
50%,4.00,3954.00,677.00,29.00,988.50,3.00
75%,6.00,4178.00,746.00,34.00,1044.50,4.00
max,8.00,4906.00,795.00,76.00,1226.50,8.00


## Création des chunks de phrases. 
On parcours les toutes les pages des pdfs. Pour chaque page, on prends tout le texte on le sépare en phrases qu'on met dans une liste, on crée ensuite une liste donc chaque élément est une liste de 10 phrases. On met ca dans un dictionnaire avec les données suivante: le nom du papier, le numéro de page absolue du papier, le lien du papier (local). On fait une liste du dictionnaire pour toutes les pages d'un papier. On rajoute dans cette liste tout les autres pdfs. Ca nous donne un gros dataframe avec un ensemble de listes de phrases. Apprès on prend ces chunks on les join et on crée u...

In [38]:
def pdfs_to_chunks(data_path,pdfs_pages_text_list):
    # pdf_files = ["data/"+f for f in os.listdir(data_path) if f.endswith('.pdf')]
    chunks = []
    # pdf_and_text,pdfs_pages_text_list = open_pdfs(pdf_files)
    
    for (pdf_num,pdf) in enumerate(pdfs_pages_text_list):
        for page in pdf:
            for chunk in page["chunks"]:
                chk = {}
                chk["pdf_number"] = pdf_num
                chk["page_number"] = page["page_number"]
                chk["text"] = chunk
                chk["chunk_count"] = len(chk["text"])
                chk["chunk_token_count"] = len(chk["text"])/4

                chunks.append(chk)

    return chunks
            
chunks_list = pdfs_to_chunks("dddaz",pdfs_pages_text_list)
df = pd.DataFrame(chunks_list)
df.describe().round(2)
    

,pdf_number,page_number,chunk_count,chunk_token_count
count,7825.00,7825.00,7825.00,7825.00
mean,45.08,75.35,979.19,244.80
std,33.16,141.44,590.59,147.65
min,0.00,0.00,1.00,0.25
25%,7.00,5.00,515.00,128.75
50%,43.00,10.00,957.00,239.25
75%,77.00,48.00,1366.00,341.50
max,102.00,547.00,7724.00,1931.00


In [62]:
df = pd.DataFrame(chunks_list)
df.describe().round(2)
df = df[ df["chunk_token_count"] >= 20]
len(df)
df = df[df["chunk_token_count"] <= 600]
df.head()

,pdf_number,page_number,text,chunk_count,chunk_token_count
0,0,0,Playing Atari with Deep Reinforcement Learning...,1855,463.75
1,0,0,However reinforcement learning presents severa...,1327,331.75
2,0,1,Figure 1: Screen shots from ﬁve Atari 2600 Gam...,1661,415.25
3,0,1,K}. The action is passed to the emulator and m...,1184,296.00
4,0,1,"As a result, we can apply standard reinforceme...",1107,276.75
